# 0. Setup
Import dependencies.

In [1]:
import json
from datetime import date
from datetime import datetime
from dateutil.relativedelta import relativedelta
import requests
from config import api_key

# 1. Initialize Variables
- `today`: Today's date.

## URLs
- `discover_base_url`: Base URL of TMDB's Discover API to search movies by different types of data. This includes the API key.
- `movie_base_url`: Base URL of TMDB's Movie API to search details of a specific movie by ID.
- `discover_target_url`: Full URL of TMDB's Discover API which includes TMDB properties to filter movies by. Example URL: https://api.themoviedb.org/3/discover/movie?api_key={api_key}&release_date.gte=2010-01-01&release_date.lte=2010-01-31&region=us&with_release_type=2|3&vote_count.gte=1&sort_by=popularity.desc&page=1
- `movie_target_url`: Full URL of TMDB's Movie API which includes the TMDB movie. Example URL: https://api.themoviedb.org/3/movie/550?api_key={api_key}

## TMDB Properties
- `release_date.gte={release_start_date}`: Earliest date of movies' release dates to search for using TMDB's Discover API.
- `release_date.lte={today}`: Latest date of movies' release dates to search for using TMDB's Discover API.

**Hard-Coded Properties**:
- `region=US`: ISO 3166-1 code to filter movies' release dates by. Set specifically to *US*.
- `with_release_type=2|3`: Type of releases. *2|3* pulls movies with theatrical releases.
- `vote_count.gte=1`: Minimum number of votes by TMDB users on movies to search for in TMDB's Discover API. Set to *1* to exclude movies with 0 votes, reducing overall search results.
- `sort_by=popularity.desc`: Sort movies pulled through TMDB's Discover API by [TMDB's popularity metric](https://developers.themoviedb.org/3/getting-started/popularity) among TMDB users, descending.

In [24]:
today = date.today()

discover_base_url = f"https://api.themoviedb.org/3/discover/movie?api_key={api_key}"
movie_base_url = "https://api.themoviedb.org/3/movie/"

# Initialize start date as 10 years from today. Keep first day of month.
release_start_date = date((today - relativedelta(years = 10)).year, today.month, 1)
region = "US"

# Append TMDB properties to Discover base URL.
discover_target_url = f"{discover_base_url}&release_date.gte={str(release_start_date)}&release_date.lte={str(today)}&region=US&with_release_type=2|3&vote_count.gte=1&sort_by=popularity.desc"

# Initialize variables of empty lists to populate upon iterating through APIs.
movies_lst = []
directors_lst = []
actors_lst = []

In [25]:
print(f"Request: {str(release_start_date)} to {str(today)}")

for page in range(1, 500):
    discover_target_url = f"{discover_base_url}&release_date.gte={str(release_start_date)}&release_date.lte={str(today)}&region=US&with_release_type=2|3&vote_count.gte=1&sort_by=popularity.desc&page={page}"

    print(f"Page {page} of 500:")

    discover_response = requests.get(discover_target_url, timeout=None)
    discover_data = discover_response.json()
    results = discover_data['results']

    for movie in results:
        id = movie["id"]
        movie_target_url = f"{movie_base_url}{id}?api_key={api_key}&append_to_response=credits"

        movie_response = requests.get(movie_target_url, timeout=None)
        movie_data = movie_response.json()
        movie_release_date = datetime.strptime(movie_data['release_date'], '%Y-%m-%d').date()

        if (movie_data['budget'] == 0 and movie_data['revenue'] == 0):
            # print(f"{id} ({movie_data['title']}) has both $0 budget and revenue. Skipping...")
            continue
        elif movie_data['original_language'] != 'en':
            # print(f"{id} ({movie_data['title']}) is not originally in English. Skipping...")
            continue
        elif (movie_release_date < release_start_date or movie_release_date > today):
            # print(f"{id} ({movie_data['title']}) was released outside of the {release_start_date} to {today} range. Skipping...")
            continue
        else:
            movies_lst.append({
                "id": id,
                "title": movie_data['title'],
                "tagline": movie_data['tagline'],
                "overview": movie_data['overview'],
                "homepage": movie_data['homepage'],
                "release_date": movie_data['release_date'],
                "budget": movie_data['budget'],
                "revenue": movie_data['revenue'],
                "popularity": movie_data['popularity'],
                "runtime": movie_data['runtime'],
                "poster_path": movie_data['poster_path']
            })
            print(f"{movie_data['title']} was successfully added!")
            
            for actor in movie_data['credits']['cast']:
                if actor['known_for_department'] == "Acting":
                    actors_lst.append({
                        "id": id,
                        "name": actor['name'],
                        "character": actor['character'],
                        "profile_path": actor['profile_path']
                    })

            for member in movie_data['credits']['crew']:
                if member['known_for_department'] != 'Director':
                    continue
                elif member['known_for_department'] == 'Director':
                    directors_lst.append({
                        "id": id,
                        "name": member['name'],
                        "profile_path": member['profile_path']
                    })

Request: 2013-01-01 to 2023-01-12
Page 1 of 500:
Puss in Boots: The Last Wish was successfully added!
Avatar: The Way of Water was successfully added!
Violent Night was successfully added!
M3GAN was successfully added!
Black Adam was successfully added!
Glass Onion: A Knives Out Mystery was successfully added!
The Woman King was successfully added!
Strange World was successfully added!
Roald Dahl's Matilda the Musical was successfully added!
Prey for the Devil was successfully added!
Black Panther: Wakanda Forever was successfully added!
Guillermo del Toro's Pinocchio was successfully added!
Medieval was successfully added!
Fall was successfully added!
Terrifier 2 was successfully added!
The Menu was successfully added!
Page 2 of 500:
Top Gun: Maverick was successfully added!
Smile was successfully added!
Monstrous was successfully added!
Lyle, Lyle, Crocodile was successfully added!
Spider-Man: No Way Home was successfully added!
Jurassic World Dominion was successfully added!
Jeepers

In [4]:
print(f"Total number of movies added: {len(movies_lst)}")

Total number of movies added: 5292


In [18]:
movies_lst[5291]

{'id': 433502,
 'title': 'The Aftermath',
 'tagline': '',
 'overview': 'In the aftermath of World War II, a British colonel and his wife are assigned to live in Hamburg during the post-war reconstruction, but tensions arise with the German widower who lives with them.',
 'homepage': 'http://www.foxsearchlight.com/theaftermath',
 'release_date': '2019-03-01',
 'budget': 0,
 'revenue': 9215591,
 'popularity': 15.256,
 'runtime': 108,
 'poster_path': '/slGZ8miznmAZAaDHEz6BKo7f1N5.jpg'}

In [ ]:
# Store lists of movies and directors/actors into Postgres as separate tables.